In [3]:
# huggingface_hub integration
from dust3r.model import AsymmetricCroCo3DStereo
import torch

model = AsymmetricCroCo3DStereo.from_pretrained("nielsr/DUSt3R_ViTLarge_BaseDecoder_512_dpt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:  54%|#####4    | 1.24G/2.28G [00:00<?, ?B/s]

AsymmetricCroCo3DStereo(
  (patch_embed): PatchEmbedDust3R(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (mask_generator): RandomMask()
  (rope): RoPE2D()
  (enc_blocks): ModuleList(
    (0-23): 24 x Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
        (rope): RoPE2D()
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop2): Dropout(p

## Office frame 0-5

In [2]:
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import csv

if __name__ == '__main__':
    device = 'cpu'
    batch_size = 1
    schedule = 'cosine'
    lr = 0.01
    niter = 300

    model_name = "naver/DUSt3R_ViTLarge_BaseDecoder_512_dpt"
    model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)
    
    for i in range(0, 5):
        images = load_images(["office_frames/frame%d.jpg" % i, "office_frames/frame%d.jpg" % (i+1)], size=512)

        pairs = make_pairs(images, scene_graph='complete', prefilter=None, symmetrize=True)
        output = inference(pairs, model, device, batch_size=batch_size)

        # at this stage, you have the raw dust3r predictions
        view1, pred1 = output['view1'], output['pred1']
        view2, pred2 = output['view2'], output['pred2']

        output = 'img%d.csv' % i
        # Open the file in write mode
        with open(output, mode='w', newline='') as file:

            writer = csv.writer(file)
            writer.writerow(['x', 'y', 'z', 'r', 'g', 'b'])

            # Write each row of the matrix to the CSV file
            for r, row in enumerate(pred1['pts3d'][0]):
                for c, element in enumerate(row):
                    data = element.tolist()

                    data.append(view1['img'][0][0, r, c].tolist())
                    data.append(view1['img'][0][1, r, c].tolist())
                    data.append(view1['img'][0][2, r, c].tolist())

                    writer.writerow(data)

        print(f"Matrix has been written to {output}")

        output = 'img%d.csv' % (i+1)

        # Open the file in write mode
        with open(output, mode='w', newline='') as file:

            writer = csv.writer(file)
            writer.writerow(['x', 'y', 'z', 'r', 'g', 'b'])

            # Write each row of the matrix to the CSV file
            for r, row in enumerate(pred2['pts3d_in_other_view'][0]):
                for c, element in enumerate(row):
                    data = element.tolist()

                    data.append(view2['img'][0][0, r, c].tolist())
                    data.append(view2['img'][0][1, r, c].tolist())
                    data.append(view2['img'][0][2, r, c].tolist())

                    writer.writerow(data)

        print(f"Matrix has been written to {output}")



>> Loading a list of 2 images
 - adding office_frames/frame0.jpg with resolution 1080x1920 --> 288x512
 - adding office_frames/frame1.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:23<00:00, 11.65s/it]


Matrix has been written to img0.csv
Matrix has been written to img1.csv
>> Loading a list of 2 images
 - adding office_frames/frame1.jpg with resolution 1080x1920 --> 288x512
 - adding office_frames/frame2.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.65s/it]


Matrix has been written to img1.csv
Matrix has been written to img2.csv
>> Loading a list of 2 images
 - adding office_frames/frame2.jpg with resolution 1080x1920 --> 288x512
 - adding office_frames/frame3.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


...

Matrix has been written to img2.csv
Matrix has been written to img3.csv
>> Loading a list of 2 images
 - adding office_frames/frame3.jpg with resolution 1080x1920 --> 288x512
 - adding office_frames/frame4.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:22<00:00, 11.29s/it]


Matrix has been written to img3.csv
Matrix has been written to img4.csv
>> Loading a list of 2 images
 - adding office_frames/frame4.jpg with resolution 1080x1920 --> 288x512
 - adding office_frames/frame5.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:22<00:00, 11.30s/it]


Matrix has been written to img4.csv
Matrix has been written to img5.csv


## Gate frames 0-6

In [1]:
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import csv

if __name__ == '__main__':
    device = 'cpu'
    batch_size = 1
    schedule = 'cosine'
    lr = 0.01
    niter = 300

    model_name = "naver/DUSt3R_ViTLarge_BaseDecoder_512_dpt"
    model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)
    
    for i in range(1, 7):
        images = load_images(["gate_frames/frame0.jpg", "gate_frames/frame%d.jpg" % i], size=512)

        pairs = make_pairs(images, scene_graph='complete', prefilter=None, symmetrize=True)
        output = inference(pairs, model, device, batch_size=batch_size)

        view1, pred1 = output['view1'], output['pred1']
        view2, pred2 = output['view2'], output['pred2']

        # only in the first iteration, never overwrite
        if i == 1:
            output = 'gate_frames_csv/img0.csv'
            with open(output, mode='w', newline='') as file:

                writer = csv.writer(file)
                writer.writerow(['x', 'y', 'z', 'r', 'g', 'b'])

                # Write each row of the matrix to the CSV file
                for r, row in enumerate(pred1['pts3d'][0]):
                    for c, element in enumerate(row):
                        data = element.tolist()

                        data.append(view1['img'][0][0, r, c].tolist())
                        data.append(view1['img'][0][1, r, c].tolist())
                        data.append(view1['img'][0][2, r, c].tolist())

                        writer.writerow(data)

            print(f"Matrix has been written to {output}")

        output = 'gate_frames_csv/img%d.csv' % (i)

        # Open the file in write mode
        with open(output, mode='w', newline='') as file:

            writer = csv.writer(file)
            writer.writerow(['x', 'y', 'z', 'r', 'g', 'b'])

            # Write each row of the matrix to the CSV file
            for r, row in enumerate(pred2['pts3d_in_other_view'][0]):
                for c, element in enumerate(row):
                    data = element.tolist()

                    data.append(view2['img'][0][0, r, c].tolist())
                    data.append(view2['img'][0][1, r, c].tolist())
                    data.append(view2['img'][0][2, r, c].tolist())

                    writer.writerow(data)

        print(f"Matrix has been written to {output}")



Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead
>> Loading a list of 2 images
 - adding gate_frames/frame0.jpg with resolution 1080x1920 --> 288x512
 - adding gate_frames/frame1.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.57s/it]


Matrix has been written to gate_frames_csv/img0.csv
Matrix has been written to gate_frames_csv/img1.csv
>> Loading a list of 2 images
 - adding gate_frames/frame0.jpg with resolution 1080x1920 --> 288x512
 - adding gate_frames/frame2.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.67s/it]


Matrix has been written to gate_frames_csv/img2.csv
>> Loading a list of 2 images
 - adding gate_frames/frame0.jpg with resolution 1080x1920 --> 288x512
 - adding gate_frames/frame3.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


...

Matrix has been written to gate_frames_csv/img4.csv
>> Loading a list of 2 images
 - adding gate_frames/frame0.jpg with resolution 1080x1920 --> 288x512
 - adding gate_frames/frame5.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.93s/it]


Matrix has been written to gate_frames_csv/img5.csv
>> Loading a list of 2 images
 - adding gate_frames/frame0.jpg with resolution 1080x1920 --> 288x512
 - adding gate_frames/frame6.jpg with resolution 1080x1920 --> 288x512
 (Found 2 images)
>> Inference with model on 2 image pairs


100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 10.95s/it]


Matrix has been written to gate_frames_csv/img6.csv


In [ ]:
import open3d as o3d
import pandas as pd
import numpy as np

# Function to load a point cloud from a CSV file
def foo(i):
    csv_file = f'gate_frames_csv/img{i}.csv'  # Use f-string for dynamic file name
    df = pd.read_csv(csv_file)

    # Assuming your CSV file has columns named 'x', 'y', 'z', 'r', 'g', 'b'
    points = df[['x', 'y', 'z']].values
    colors = df[['r', 'g', 'b']].values

    # Create and return a PointCloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    return point_cloud

point_clouds = [foo(i) for i in range(7)]
o3d.visualization.draw_geometries(point_clouds)
